In [2]:
pip install praw

     ------------------------------------ 191.0/191.0 kB 399.0 kB/s eta 0:00:00
     ---------------------------------------- 56.9/56.9 kB 1.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import praw
import csv

In [4]:
reddit = praw.Reddit(
    client_id="eIGc4dlNt2ApFT-IoqaufA",
    client_secret="fEEIU4jHjNEpU2vjweqt6dEuKq3QZQ",
    user_agent="Infinite_Clock467",
    redirect_uri="http://localhost",
)

In [9]:
submission = reddit.submission(id='nsm98m')
submission.comments.replace_more(limit=None)
[]
for comment in submission.comments.list():
    print(comment.body + '\n')

The Murder of Roger Ackroyd by Agatha Christie still has the best twist I’ve ever read.

{{Magpie Murders}} by Antony Horowitz. Halfway through when I realized what was going on I had a mini freak out because it's such and incredible book.

Anything by Agatha Christie 

*A Murder of Quality* by John le Carré made me think of Christie, as well.

The Silent Patient

Searching "Good murder mysteries that really draw you in with great twists" on librimood gave a bunch of Agatha Christie suggestions as well as "An Unwanted Guest" by Shari Lapena

I think you would enjoy Presumed Innocent" by Scott Turow.  So well written and such an intriguing twisty story.   You will love it.  Won multiple awards.

{{ The Last Policeman }}

Chestnut man! So creepy but so thrilling

Commenting so I can come back to this later

{The Guest List by Lucy Foley}

{Murder on the Orient Express by Agatha Christie}

{the seven deaths of evelyn hardcastle}

The Dain Curse by Dashiell Hammett

The last great mystery 

In [ ]:
search_query = "GPT"
num_posts_to_fetch = 5  # You can adjust this value to fetch more posts

# Fetch the posts using the search query
posts = reddit.subreddit("all").search(search_query, limit=num_posts_to_fetch)

# Prepare the data for CSV
data = []
for post in posts:
    post_title = post.title
    post_url = post.url

    # Fetch comments from the post
    post.comments.replace_more(limit=None)
    for comment in post.comments.list():
        comment_body = comment.body
        data.append([post_title, post_url, comment_body])

# Write data to a CSV file
csv_filename = "reddit_gpt_comments.csv"
with open(csv_filename, "w", newline="", encoding="utf-8") as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(["Post Title", "Post URL", "Comment"])
    csv_writer.writerows(data)

print(f"Posts and comments related to 'GPT' saved to '{csv_filename}'.")


In [8]:
search_query = "ChatGPT OR Chat GPT accuracy OR ChatGPT user experience"
num_posts_to_fetch = 5

posts = reddit.subreddit("all").search(search_query, limit=num_posts_to_fetch)

comments = []
for post in posts:
    post.comments.replace_more(limit=None)  # Retrieve all comments (including nested)
    for comment in post.comments.list():
        comments.append(comment.body)

data = []
for comment in comments:
    submission_id = comment.link_id.split("_")[-1]
    submission = reddit.submission(id=submission_id)
    post_title = submission.title
    comment_body = comment.body
    data.append([post_title, comment_body])

csv_filename = "reddit_comments.csv"
with open(csv_filename, "w", newline="", encoding="utf-8") as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(["Post Title", "Comment"])
    csv_writer.writerows(data)

print(f"Comments saved to '{csv_filename}'.")

AttributeError: 'str' object has no attribute 'link_id'